In [187]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re
import time

In [195]:
class MinticETL(object):
    #Declaracion de variables
    now = datetime.now()
    fecha_hoy = str(now.strftime("%Y.%m.%d"))
    ahora_format = "%Y-%m-%d"'T'"%H:%M:%S"
    ahora = str(now.strftime(ahora_format))
    ahora_cdd = str(now.strftime("%Y-%m-%d"' '"%H:%M:%S"))
    fechaAhora = str(now.strftime("%Y%m%d%H%M%S"))
    datos_logs =""
    
    def __init__(self):
        #Inicializar las variables de la instancia
        pass

    def getElasticConnect(self, timeout=60, max_retries=3, retry_on_timeout=True):
        context = create_default_context(cafile=parametros.cafile)
        es = Elasticsearch(
            parametros.servidor,
            http_auth=(parametros.usuario_EC, parametros.password_EC),
            scheme="https",
            port=parametros.puerto,
            ssl_context=context,
            timeout=60, max_retries=3, retry_on_timeout=True
        ) 
        return es

    def getDataIndice(self
                      ,indice
                      ,jerarquia
                      ,source=["*"]
                      ,default_fec_eject='2021-04-01 00:00:00'):
        # aqui se busca la fecha ultima de control de Estado Cambuim  
        # parametros.mintic_control=dev-control_ejecucion_mintic
        # en vez de  indice = parametros.mintic_concat_index que es = dev-mintic-concat-2021.06.24
        # me imagino que es donde guardan todos los controles de las ejecuciones  * devuelvo el cambio *
        total_docs = 1
        try:
            es = getElasticConnect()
            response = {"hits":{"hits":[]}}
            response = es.search(
                index=  indice,     # ***  quite esto, creo que no va, pero lo devolvi,habia puesto indice
                body={
                   "_source": source,
                      "query": {
                        "bool": {
                          "filter": [
                          {
                            "exists": {
                              "field":jerarquia
                            }
                          }
                      ]
                    }
                  }
                },
                size=total_docs
            )
            #print(es.info())
            elastic_docs = response["hits"]["hits"]
            fields = {}
            for num, doc in enumerate(elastic_docs):
                fecha_ejecucion = doc["_source"]['gestion.fechaControl']
        except:
            response["hits"]["hits"] = []
            
        if response["hits"]["hits"] == []:    
            fecha_ejecucion = default_fec_eject   # el formato tiene que ser solo numeros, asi aparecen los registros[GM]

        return fecha_ejecucion

    def drop_invalid_values(self, df, fields=[], values = ['NA','',np.NaN,None],operator='or'):
        #Elimina los los registros con valores invalidos en las columnas seleccionadas
        #Retorna un dataframe con los registros eliminados
        fields = df.columns if fields==[] else fields
        if operator=='or':
            freduce = np.logical_or.reduce
        elif operator=='and':
            freduce = np.logical_and.reduce
        else:
            return
        df.drop(df[freduce([df[c].isin(values) for c in fields])].index,inplace=True)
        
        return df[df[freduce([df[c].isin(values) for c in fields])].index]
    
    def filterKeys(self, document, use_these_keys):
        return {key: document.get(key) for key in use_these_keys }

    def getDatosSemilla(self,es, total_docs=10000):
        try:
            response = es.search(
                index= parametros.semilla_inventario_index,
                body={
                       "_source": ['site_id'
                                   ,'nombre_municipio'
                                   , 'nombre_departamento'
                                   , 'nombre_centro_pob'
                                   ,'energiadesc'
                                   ,'nombreSede'
                                   ,'latitud'
                                   , 'longitud'
                                   ,'id_Beneficiario'
                                   ,'COD_ISO'
                                   ,'codDanesede'
                                   ,'cod_servicio'
                                   ,'codDaneMuni'
                                   ,'nombre_centro_pob'
                                   ,'codCentroPoblado'
                                   ,'codDaneInstitucionEdu'
                                   ,'tipoSitio'
                                   ,'detalleSitio'
                                   ,'energia'
                                   ,'region'
                                   ,'matricula'
                                   ,'DDA'
                                   ,'grupoDesc'
                                   ,'estadoInstalacion'
                                  ]
                },
                size=total_docs
            )

            elastic_docs = response["hits"]["hits"]

            datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])

        except Exception as e:
            print(e.message)
            print("fecha:",now,"- Error en lectura de datos semilla")
            datos_semilla = ['nombreSede', 'DDA', 'estadoInstalacion', 'latitud', 'COD_ISO',
                            'energia', 'nombre_departamento', 'codCentroPoblado', 'longitud',
                            'codDanesede', 'tipoSitio', 'codDaneMuni', 'nombre_centro_pob',
                            'site_id', 'matricula', 'energiadesc', 'grupoDesc', 'cod_servicio',
                            'region', 'detalleSitio', 'nombre_municipio', 'id_Beneficiario',
                            'codDaneInstitucionEdu']
            
        return datos_semilla
            
    def getLocation(self,loc,latlon='lat', lat_def='4.596389', lon_def='-74.074639'):
        patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
        loc=str(loc)
        if (not patron.match(loc) is None) and (loc!=''):
            resp = loc.replace(',','.')
        else:
            #Código a ejecutar si las coordenadas no son válidas
            resp = lat_def if latlon=='lat' else lon_def
        return resp
    
    def getIndiceCurado(indice, source, fecha_max, fecha_tope, total_docs = 10000):
        total_docs = 10000
        try:
            response = es.search(
                index= indice,
                body={
                    "_source": source
                    ,"query":{
                        "bool": {
                          "filter": [
                            {
                              "range": {
                                "registration_date": {
                                  "gte": fecha_max,
                                  "lt": fecha_tope
                                }
                              }
                            }
                          ],

                        }
                    }
                },
                size=total_docs
            )
            elastic_docs = response["hits"]["hits"]
        except:
            elastic_docs = {}
            
        return elastic_docs

In [197]:
etl = MinticETL()

In [198]:
etl.getElasticConnect()

<Elasticsearch([{'host': 'hotzone3'}])>

In [199]:
etl.fechaAhora

'20210715165601'

In [192]:
etl.getDataIndice(indice='dev-control_ejecucion_mintic',jerarquia="jerarquia_gestion_estado_incidentes")

'2021-04-01T01:40:00'

In [193]:
semilla=etl.getDatosSemilla(etl.getElasticConnect())
semilla

,nombreSede,DDA,estadoInstalacion,latitud,COD_ISO,energia,nombre_departamento,codCentroPoblado,longitud,codDanesede,...,site_id,matricula,energiadesc,grupoDesc,cod_servicio,region,detalleSitio,nombre_municipio,id_Beneficiario,codDaneInstitucionEdu
0,C.E.R LA PAULINA,5,3,"6,7048548",CO-ANT,1,ANTIOQUIA,333,"-76,14525469",205854000501,...,22154-ZGYO176,111,RED INTERCONECTADA,OBLIGATORIOS,ZGYO176,A,8,VALDIVIA,22154,205854000501
1,CENTRO EDUCATIVO RURAL EL CERRO,5,3,"4,225166667",CO-ANT,1,ANTIOQUIA,333,"-73,10716667",205284000201,...,20022-ZGYO177,22,RED INTERCONECTADA,OBLIGATORIOS,ZGYO177,A,8,FRONTINO,20022,105284000214
2,C. E. R. MERCEDES YEPES,5,1,"6,93848274",CO-ANT,1,ANTIOQUIA,333,"-75,151177190",205040000161,...,18787-,42,RED INTERCONECTADA,OBLIGATORIOS,,A,8,ANORÍ,18787,205040000357
3,C. E. R. MEDIAS FALDAS,5,1,"7,04797951",CO-ANT,4,ANTIOQUIA,333,"-75,19442611",205040000616,...,18802-,84,TIENE ENERGÍA,Opción 2,,A,8,ANORÍ,18802,105040000212
4,C. E. R. ROSAURA ORTEGA,5,2,"7,06272584",CO-ANT,1,ANTIOQUIA,333,"-75,22730972",205040000641,...,18804-ZGYO583,14,RED INTERCONECTADA,OBLIGATORIOS,ZGYO583,A,8,ANORÍ,18804,205040000357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,ESCUELA SANTA ISABEL,5,1,"6,39026335",CO-VAU,3,VAUPÉS,333,"-73,138349330",297001002239,...,67733-,53,SOLUCIÓN SOLAR,OBLIGATORIOS,,A,8,PACOA,67733,297001001879
6943,ESCUELA RURAL DE SAN LUIS DE PIEDRA ÑI,5,1,"0,08164385",CO-VAU,4,VAUPÉS,333,"-70,07416384",297511000036,...,70622-,14,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70622,297001001879
6944,ESCUELA RURAL DE SANTA ROSA,5,1,,CO-VAU,4,VAUPÉS,333,,297511000044,...,70623-,36,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70623,297001001879
6945,ESCUELA RURAL DE TOAKA,5,1,"1,007644476",CO-VAU,4,VAUPÉS,333,"-71,25103723",297511000028,...,70624-,32,SIN INFORMACIÓN,OBLIGATORIOS,,A,8,PACOA,70624,297001001879


In [194]:
etl.getLocation('4596389')

'4.596389'